In [66]:
import torch
import timm
from PIL import Image
from urllib.request import urlopen
import requests
import numpy as np
import torchvision as tivision

In [67]:
import torchattacks

In [68]:
from transformers import AutoImageProcessor, ResNetForImageClassification

In [69]:
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
imagenet_labels = requests.get(url).json()

In [70]:
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
raw_model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

In [71]:
class DoubleModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model.to(dtype=torch.double).cuda()

    def forward(self, x):
        x = x.to(dtype=torch.double)
        outputs = self.model(x)
        logits = outputs.logits.to(dtype=torch.double)
        return logits

In [72]:
wrapped_model = DoubleModelWrapper(raw_model)
wrapped_model.eval()

DoubleModelWrapper(
  (model): ResNetForImageClassification(
    (resnet): ResNetModel(
      (embedder): ResNetEmbeddings(
        (embedder): ResNetConvLayer(
          (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
        )
        (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      )
      (encoder): ResNetEncoder(
        (stages): ModuleList(
          (0): ResNetStage(
            (layers): Sequential(
              (0): ResNetBottleNeckLayer(
                (shortcut): ResNetShortCut(
                  (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                )
                (layer): Sequential(
           

In [73]:
image_url = 'https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg'
img = Image.open(urlopen(image_url))
image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor(),
        tivision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
img_tensor = image_transforms(img).unsqueeze(0).cuda()

CW

In [74]:
def attack_model_with_cw( image_url,model, label=9):
    img = Image.open(urlopen(image_url))

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor(),
        # tivision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    img_tensor = image_transforms(img).unsqueeze(0).cuda()
    imagenet_labels_tensor = torch.tensor([label]).cuda()

   
    atk = torchattacks.CW(model, c=1, kappa=0, steps=20, lr=0.1)
    raw_adversarial = atk(img_tensor, imagenet_labels_tensor)
    return raw_adversarial

In [75]:
print("Original image")
print("Prediction:", imagenet_labels[torch.argmax(wrapped_model(img_tensor)[0]).item()])


adv_images = attack_model_with_cw( 'https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg',wrapped_model)

print("Adversarial image")
print("Prediction:", imagenet_labels[torch.argmax(wrapped_model(adv_images)[0]).item()])

Original image
Prediction: ostrich
Adversarial image
Prediction: bustard


EADEN

In [76]:
def attack_model_with_eaden( image_url, model, label=9):
    img = Image.open(urlopen(image_url))

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor(),
        tivision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    img_tensor = image_transforms(img).unsqueeze(0).cuda()
    imagenet_labels_tensor = torch.tensor([label]).cuda()

    atk = torchattacks.EADEN(model,binary_search_steps=10 ,max_iterations=10)
    raw_adversarial = atk(img_tensor, imagenet_labels_tensor)
    return raw_adversarial

In [77]:

print("Original image")
print("Prediction:", imagenet_labels[torch.argmax(wrapped_model(img_tensor)[0]).item()])

adv_images = attack_model_with_eaden( 'https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg',wrapped_model)


print("Adversarial image")
print("Prediction:", imagenet_labels[torch.argmax(wrapped_model(adv_images)[0]).item()])

print(torch.max(torch.nn.functional.softmax(wrapped_model(adv_images), dim=1)).cpu().detach().numpy())

Original image
Prediction: ostrich
Adversarial image
Prediction: bustard
0.9999280500692641


SUMMARY

In [78]:
import time
import pandas as pd

In [79]:
def benchmark_methods(methods: dict, image_url: str, model, num_of_iters = 5):
    """
    Benchmark a set of adversarial attack methods.

    Args:
        methods (dict): Dictionary of attack methods {method_name: function}.
        image_url (str): URL of the input image.
        model: Pretrained model to attack.

    Returns:
        pd.DataFrame: Benchmarking results as a tidy table.
    """
    results = []
    
    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor(),
        tivision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    for method_name, method in methods.items():
        times = []
        probabilities_before = []
        probabilities_after = []
        label_before = []
        label_after = []
        # successes = []

        print(f"Benchmarking {method_name}...")

        for _ in range(num_of_iters):  # Run each method 3 times
            start_time = time.time()
            
            # Run the adversarial method
            outputs = method(image_url, model)
            raw_adversarial = outputs
            
            img = Image.open(urlopen(image_url))
            input_tensor = image_transforms(img).unsqueeze(0).cuda()
            
            
            # Measure time
            elapsed_time = time.time() - start_time
            times.append(elapsed_time)
            
            # Collect probabilities
            with torch.no_grad():
                # For the original input
                logits_before = model(input_tensor)
                probabilities_before_run = torch.nn.functional.softmax(logits_before, dim=1)
                probabilities_before.append(torch.max(probabilities_before_run).cpu().detach().numpy())
                label_before.append(int(torch.argmax(logits_before).cpu().detach().numpy()))
                
                

                # For the adversarial input
                logits_after = model(raw_adversarial)
                probabilities_after_run = torch.nn.functional.softmax(logits_after, dim=1)
                probabilities_after.append(torch.max(probabilities_after_run).cpu().detach().numpy())
                label_after.append(int(torch.argmax(logits_after).cpu().detach().numpy()))
                
            
            # Track success
            # successes.append(success)
        
        # Store results for this method
        results.append({
            "Method": method_name,
            "Avg Time (s)": np.mean(times),
            "Avg Prob Before": np.mean(probabilities_before),
            "Avg Prob After": np.mean(probabilities_after),
            "Predicted Label Before": label_before,
            "Predicted Label After": label_after,
            # "Success Rate": np.mean(successes),
        })

    # Convert to DataFrame for a tidy table
    return pd.DataFrame(results)

In [80]:
methods = {
    "EAD_EN": attack_model_with_eaden,
    "CW": attack_model_with_cw,
    
}

image_url ='https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg'  # Replace with actual image path

benchmark_results = benchmark_methods(methods, image_url, wrapped_model, num_of_iters=5)

display(benchmark_results)

Benchmarking EAD_EN...
Benchmarking CW...


,Method,Avg Time (s),Avg Prob Before,Avg Prob After,Predicted Label Before,Predicted Label After
0,EAD_EN,6.407542,0.999963,0.999928,"[9, 9, 9, 9, 9]","[138, 138, 138, 138, 138]"
1,CW,0.822687,0.999963,0.975993,"[9, 9, 9, 9, 9]","[138, 138, 138, 138, 138]"
